# 采集地址：https://www.baidu.com/s?wd=%E6%98%8E%E6%98%9F&rsv_spt=1&rsv_iqid=0xbe4b76860031fb66&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=&tn=baiduhome_pg&ch=&rsv_enter=1&rsv_dl=ib&inputT=2978

In [4]:
import time,csv,os,re
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from lxml import etree


In [5]:
url='https://www.baidu.com/s?wd=%E6%98%8E%E6%98%9F&rsv_spt=1&rsv_iqid=0xbe4b76860031fb66&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=&tn=baiduhome_pg&ch=&rsv_enter=1&rsv_dl=ib&inputT=2978'
driver=webdriver.Chrome()
driver.get(url)
wait = WebDriverWait(driver,10)
driver.find_element_by_xpath('//*[@id="1"]/div/div[1]/div[1]/div[2]/p[1]/span[4]').click()
driver.find_element_by_xpath('//*[@id="1"]/div/div[1]/div[1]/div[2]/p[2]/span[3]').click()
time.sleep(5)

# 构建储存容器

In [ ]:
allname=[] #存放名字
allpic=[]  #存放图片链接
allhref=[] #存放全部链接
baidu="https://www.baidu.com" 

In [9]:
while True:
    try:
        html=etree.HTML(driver.page_source,etree.HTMLParser())
        names=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[2]/a/text()')
        ###获取这一页明星名字
        pictureurl=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[1]/a/img/@src')
        ###获取这一页明星图片
        hrefs=html.xpath('//*[@id="1"]/div/div[1]/div[2]/div[1]/div/p[1]/a/@href')
        ###获取这一页明星链接
        
        
        for i in range(len(names)):
            allname.append(names[i])
            allpic.append(pictureurl[i])
            allhref.append(baidu+hrefs[i])
        next_btn=wait.until(EC.element_to_be_clickable(
            (By.XPATH,'//*[@id="1"]/div/div[1]/div[2]/div[2]/p/span[6]')))
        next_btn.click()
        time.sleep(1)
    except:
        print('完成')
        break
driver.close()

完成


InvalidSessionIdException: Message: invalid session id


# 构建一个储存数据库

In [10]:
data=[]
for i in range(len(allname)):
    data.append((allname[i],allpic[i],allhref[i]))

# 保存文件

In [11]:
f=open('明星.csv','w',newline='')
w=csv.writer(f)
for i in data:
    w.writerow(i)
f.close()
head={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
htmls=[]
a=1
for i in data:
    name=i[0]
    url='https://baike.baidu.com/item/'+name
    r=requests.get(url,headers=head)
    r.encoding='utf-8'
    html=r.text
    htmls.append((url,html))
    print(a)
    a+=1

# 获取明星资料

In [13]:
stardata=[]
for html in htmls:
    try:
        nm=re.search(r'<dt class="basicInfo-item name">中文名</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">中文名</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        birth=re.search(r'<dt class="basicInfo-item name">出生日期</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">出生日期</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
            if birth!='':
            if '年' in birth and '月' in birth and '日' in birth:
                year,month,day=re.split('[年月日]',birth)[:3]
            elif '年' in birth and '月' in birth and '日' not in birth:
                year,month=re.split('[年月]',birth)[:2]
                day=''
            elif '年' in birth and '月' not in birth and '日' not in birth:
                year=re.split('[年]',birth)[0]
                month=''
                day=''
            elif '年' not in birth and '月' in birth and '日' in birth:
                year=''
                month,day=re.split('[月日]')[:2]
            elif '年' not in birth and '月' in birth and '日' not in birth:
                year=''
                month=re.split('[月]')[0]
                day=''
            elif '年' not in birth and '月' not in birth and '日' not in birth:
                year=''
                month=''
                day=''
            else:
                continue
        else:
            year=''
            month=''
            day=''
        height=re.search(r'<dt class="basicInfo-item name">身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        if ('CM' in height) or ('cm' in height):
            height=height[:-2]
        jiguan=re.search(r'<dt class="basicInfo-item name">出生地</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">出生地</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        xingzuo=re.search(r'<dt class="basicInfo-item name">星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]).group(1).strip() if re.search(r'<dt class="basicInfo-item name">星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>\n<dd class="basicInfo-item value">([\d\D]*?)<',html[1]) != None else ''
        if nm=='':
            continue
        stardata.append((nm,'女',html[0],year,month,day,xingzuo,html[1],jiguan,height))
    except:
        print(nm+'获取成功')

IndentationError: unexpected indent (<ipython-input-13-a16c90879ea7>, line 6)

In [15]:
for i in stardata:
    print(i[0],i[3:6])
from pony.orm import *


db = Database()


class Star(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Optional(str, column='name')
    gender = Optional(str, column='gender')
    href = Optional(str, column='href')
    year = Optional(int, column='year')
    month = Optional(int, column='month')
    day = Optional(int, column='day')
    xz = Optional(str, column='xz')
    html = Optional(str, column='html')
    address = Optional(str, column='address')
    height = Optional(float, column='height')



ModuleNotFoundError: No module named 'pony'

In [16]:
db.bind(provider='sqlite', filename=r'C:\Users\Admin\star2.sqlite')

db.generate_mapping(create_tables=True)

set_sql_debug(True)

for star in stardata:
    try:
        s=Star(name=star[0],gender=star[1],href=star[2],year=star[3],month=star[4],day=star[5],xz=star[6],html=star[7],address=star[8],height=star[9])
        db.commit()
    except:
        pass

NameError: name 'db' is not defined